In [ ]:
import warnings
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Suppressing unnecessary warnings
warnings.filterwarnings("ignore")

In [ ]:

# Check if a GPU is available, and if so, use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#torch.cuda.empty_cache()
#gc.collect()    

#tokenizer = T5Tokenizer.from_pretrained("./T5Snapshot/tokenizer_epoch5_loss1.60")
#model = T5ForConditionalGeneration.from_pretrained("./T5Snapshot/model_epoch5_loss1.60").to(device)

tokenizer = T5Tokenizer.from_pretrained("t5-base") # When training from scratch
model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)  # Move model to the GPU

#We have modify the tokenizer a little bit, since it was designed for human language only
new_tokens = ["{", "<", "}"]

# Add the tokens to the tokenizer vocabulary
tokenizer.add_tokens(new_tokens)

# Resize the model's embeddings to accommodate the extended vocabulary
model.resize_token_embeddings(len(tokenizer))

# the following 2 hyperparameters are task-specific
max_source_length = 1024  # Set to a value that covers most assembly code sequences
max_target_length = 512  # Set to a value that covers most C code sequences


#max_split_size_mb = 1024  # Adjust this value to a lower value if needed
#tokenizer.max_split_size_mb = max_split_size_mb

# Read input sequences from assembly.txt
with open("assembly1.txt", "r", encoding="utf-8") as assembly_file:
    assembly_lines = assembly_file.readlines()

# Read output sequences from functions.txt
with open("function1.txt", "r", encoding="utf-8") as functions_file:
    functions_lines = functions_file.readlines()

# Ensure that the number of lines in assembly.txt and functions.txt are the same
assert len(assembly_lines) == len(functions_lines), "Number of lines in assembly.txt and functions.txt must be the same."

# Encode inputs and targets
input_sequences = [assembly_line for assembly_line in assembly_lines]
output_sequences = [function_line for function_line in functions_lines]

input_train, input_temp, output_train, output_temp = train_test_split(
    input_sequences, output_sequences, test_size=0.1, random_state=42
)

input_test, input_val, output_test, output_val = train_test_split(
    input_temp, output_temp, test_size=0.5, random_state=42)

encoding_train = tokenizer(
    input_train,
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids_train, attention_mask_train = encoding_train.input_ids.to(device), encoding_train.attention_mask.to(device)

target_encoding_train = tokenizer(
    output_train,
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)
labels_train = target_encoding_train.input_ids.to(device) #generates the Tensor Matrix

# Replace padding token id's of the labels by -100 so it's ignored by the loss
labels_train[labels_train == tokenizer.pad_token_id] = -100


encoding_val = tokenizer(
    input_val,
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)
input_ids_val, attention_mask_val = encoding_val.input_ids.to(device), encoding_val.attention_mask.to(device)

target_encoding_val = tokenizer(
    output_val,
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)
labels_val = target_encoding_val.input_ids.to(device)

#print(tokenizer.decode(encoding_val['input_ids'][0]))
#print(input_val[0])
#print(len(tokenizer.encode(input_val[0])))
#print(output_val[0])
#print(tokenizer.decode(target_encoding_val['input_ids'][0]))

# Replace padding token id's of the labels by -100 so it's ignored by the loss
labels_val[labels_val == tokenizer.pad_token_id] = -100

print(model.device)  # Print the device the model is on to verify we use GPU
print(input_ids_train.device)  # Print the device of input_ids

# Set hyperparameters
learning_rate = 5e-5
batch_size = 16 #thumb of rule
num_epochs = 20
num_warmup_steps = 0.1 * num_epochs * len(input_sequences) / batch_size

# Define optimizer with GPU device
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Create DataLoader for training data
train_dataset = TensorDataset(input_ids_train, attention_mask_train, labels_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# DataLoader for validation set
validation_dataset = TensorDataset(input_ids_val, attention_mask_val, labels_val)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)  # No need to shuffle for validation

# Initialize lists to store training and validation losses and prepare Plot
train_losses = []
validation_losses = []
# Training loop on the GPU
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1} (Training)"):
        input_ids_batch_training, attention_mask_batch_training, labels_batch_training = batch

        # Forward pass
        loss = model(
            input_ids=input_ids_batch_training,
            attention_mask=attention_mask_batch_training,
            labels=labels_batch_training
        ).loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    train_losses.append(avg_loss)
    
    # Validation
    model.eval()
    total_validation_loss = 0.0
    for batch in tqdm(validation_dataloader, desc=f"Epoch {epoch + 1} (Validation)"):
        input_ids_batch_validation, attention_mask_batch_validation, labels_batch_validation = batch
        # Forward pass on validation data
        # Calculate the validation loss
        validation_loss = model(
            input_ids=input_ids_batch_validation,
            attention_mask=attention_mask_batch_validation,
            labels=labels_batch_validation
        ).loss

        total_validation_loss += validation_loss.item()

    # Calculate the average validation loss for this epoch
    avg_validation_loss = total_validation_loss / len(validation_dataloader)
    validation_losses.append(avg_validation_loss)
    # Plot the live loss curve
    plt.clf()
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(validation_losses, label='Validation Loss', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    #x_ticks = range(1, epoch + 1) #From 1 to Epoche
    #plt.xticks(x_ticks)
    plt.savefig('TrainValLoss.png')
    
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")
    model.save_pretrained(f"./T5Snapshot/model_epoch{epoch + 1}")
    tokenizer.save_pretrained(f"./T5Snapshot/tokenizer_epoch{epoch + 1}")

# Save the trained model
model.save_pretrained("./T5Snapshot2/model")
tokenizer.save_pretrained("./T5Snapshot/tokenizer")

**Test Set Evaluation**

In [ ]:
import Levenshtein

# Initialize lists to store results
predicted_c_codes = []
actual_c_codes = []
levenshtein_distances = []


#tokenizer = T5Tokenizer.from_pretrained("./T5Snapshot/tokenizer_")
#model = T5ForConditionalGeneration.from_pretrained("./T5Snapshot/model").to(device)
# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    for index, assembly_sequence in enumerate(input_test):
        # Encode the assembly sequence
        encoding = tokenizer(
            assembly_sequence,
            padding="longest",
            max_length=max_source_length,
            truncation=True,
            return_tensors="pt"
        )

        input_ids, attention_mask = encoding.input_ids.to(device), encoding.attention_mask.to(device)

        # Generate the corresponding C code
        output = model.generate(input_ids, max_length=max_target_length, num_return_sequences=1)
        predicted_c_code = tokenizer.decode(output[0], skip_special_tokens=True)

        # Get the actual C code
        actual_c_code = output_test[index]

        # Calculate Levenshtein distance
        levenshtein_distance = Levenshtein.distance(predicted_c_code, actual_c_code)

        # Append results to lists
        predicted_c_codes.append(predicted_c_code)
        actual_c_codes.append(actual_c_code)
        levenshtein_distances.append(levenshtein_distance)

# Calculate and print performance metrics
total_samples = len(predicted_c_codes)
avg_levenshtein_distance = sum(levenshtein_distances) / total_samples

print(f"Total samples: {total_samples}")
print(f"Average Levenshtein Distance: {avg_levenshtein_distance}")

**Inference**

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained("./T5Snapshot/tokenizer_epoch1_loss1.51",local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained("./T5Snapshot/model_epoch1_loss1.51",local_files_only=True).to(device)

input_sequence = 'endbr64 ;push   rbp;mov    rbp,rsp;mov    DWORD PTR [rbp-(1)],edi;mov    DWORD PTR [rbp-(0)],esi;mov    edx,DWORD PTR [rbp-(1)];mov    eax,DWORD PTR [rbp-(0)];add    eax,edx;pop    rbp;ret'
input_ids = tokenizer(input_sequence, return_tensors="pt").input_ids.to(device)
outputs = model.generate(input_ids,max_length=max_target_length, num_return_sequences=1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))